### **Estudo sobre decomposições matriciais - Análise da eficiência dos métodos**

---

Gabriel Oukawa <br>
Álgebra linear para ciência de dados <br>
2º Semestre de 2025

---


In [13]:
# Bibliotecas
import numpy as np
from scipy.linalg import pascal
from tabulate import tabulate
import time

In [2]:
# Precisão da máquina
eps = np.finfo(float).eps
print(eps)

2.220446049250313e-16


In [3]:
# Algoritmo "e" (estabilidade)
def calc_e(X, A):
    # Norma 2 para todos os cálculos
    A = np.linalg.pinv(A)
    cond = np.linalg.cond(A, 2)
    num = np.linalg.norm(X - A, 2)
    den = eps * np.linalg.norm(A, 2) * cond
    return num / den

# Algoritmo "res" (erro residual)
def calc_res(X, A):
    # Norma 2 para todos os cálculos
    m, n = A.shape
    identity = np.eye(m)
    num = np.linalg.norm(X @ A - identity, 2)
    den = np.linalg.norm(A, 2) * np.linalg.norm(X, 2)
    return num / den

In [23]:
# Decomposições ST e TS, seguindo os algoritmos 3.1 e 3.2
def st(A):

    n = A.shape[0]
    T = np.zeros_like(A, dtype=float)
    L = np.zeros_like(A, dtype=float)

    # t11 e a11 devem ser > 0
    t11 = np.sign(A[0, 0]) if A[0, 0] != 0 else 1.0
    if t11 * A[0, 0] <= 0:
        t11 = -t11
    T[0, 0] = t11
    L[0, 0] = np.sqrt(t11 * A[0, 0])

    for k in range(n - 1):
        Lk = L[:k+1, :k+1]
        Tk = T[:k+1, :k+1]

        ak1 = A[:k+1, k+1]
        a1k = A[k+1, :k+1]
        akk = A[k+1, k+1]

        lk1 = np.linalg.solve(Lk, Tk @ ak1)

        lhat = np.linalg.solve(Lk, a1k)

        s = akk - lhat @ lk1

        tkk = 1.0 if s > 0 else -1.0
        gamma = tkk * s
        if gamma <= 0: # gamma deve ser > 0
            break

        L[k+1, :k+1] = lk1
        L[k+1, k+1] = np.sqrt(gamma)
        T[k+1, :k+1] = np.linalg.solve(Lk.T, (lk1 - tkk * lhat))
        T[k+1, k+1] = tkk

    return T, L

def ts(A):

    n = A.shape[0]
    T = np.zeros_like(A, dtype=float)
    L = np.zeros_like(A, dtype=float)

    # t11 deve ser > 0
    t11 = np.sign(A[0, 0]) if A[0, 0] != 0 else 1.0
    if t11 * A[0, 0] <= 0:
        t11 = -t11
    T[0, 0] = t11
    L[0, 0] = np.sqrt(t11 * A[0, 0])

    for k in range(n - 1):
        Lk = L[:k+1, :k+1]
        Tk = T[:k+1, :k+1]

        ak1 = A[:k+1, k+1]
        a1k = A[k+1, :k+1]
        akk = A[k+1, k+1]

        lk1 = np.linalg.solve(Lk, np.linalg.solve(Tk, ak1))

        lhat = np.linalg.solve(Lk, a1k)

        s = akk - lhat @ lk1

        tkk = 1.0 if s > 0 else -1.0
        gamma = s / tkk
        if gamma <= 0: # gamma deve ser > 0
            break

        L[k+1, :k+1] = lk1
        L[k+1, k+1] = np.sqrt(gamma)
        T[k+1, :k+1] = np.linalg.solve(Lk.T, (lk1 - tkk * lhat))
        T[k+1, k+1] = tkk

    return T, L

# Decomposição conjugada
def cj(A):
    m, n = A.shape
    eigvals, eigvecs = np.linalg.eigh(A.T @ A)

    # Ordenar autovalores e autovetores
    idx = np.argsort(eigvals)[::-1]
    eigvals = eigvals[idx]
    eigvecs = eigvecs[:, idx]

    k = np.sum(eigvals > 1e-16)
    P = eigvecs

    gammas = np.sqrt(np.clip(eigvals[:k], 0, None))

    # Construir Q
    Qk = np.column_stack([(A @ P[:, i]) / gammas[i] for i in range(k)])

    if k < m:
        Q_rest, _ = np.linalg.qr(np.random.randn(m, m - k))

        for i in range(Q_rest.shape[1]):
            for j in range(Qk.shape[1]):
                Q_rest[:, i] -= np.dot(Qk[:, j], Q_rest[:, i]) * Qk[:, j]
            Q_rest[:, i] /= np.linalg.norm(Q_rest[:, i])
        Q = np.column_stack((Qk, Q_rest))
    else:
        Q = Qk

    G = np.zeros((m, n))
    np.fill_diagonal(G, np.concatenate([gammas, np.zeros(min(m, n) - k)]))

    return Q, G, P

In [26]:
# Testes numéricos
n_runs = [2, 3, 4, 6, 8, 10, 50, 100, 200, 500]
table = []
np.random.seed(42)

def fmt(x):
    return f"{x:.2e}"

for n in n_runs:
    A_rand = np.random.rand(n, n)
    A = A_rand + n * np.eye(n)

    condA = np.linalg.cond(A)

    t0 = time.perf_counter()
    R = np.linalg.cholesky(A.T @ A)
    X_chol = np.linalg.solve(R.T, np.linalg.solve(R, A.T)).T
    t_chol = time.perf_counter() - t0

    t0 = time.perf_counter()
    Q, R = np.linalg.qr(A, mode="reduced")
    X_qr = Q @ R
    t_qr = time.perf_counter() - t0

    t0 = time.perf_counter()
    U, S, Vt = np.linalg.svd(A, full_matrices=False)
    X_svd = U @ np.diag(S) @ Vt
    t_svd = time.perf_counter() - t0

    t0 = time.perf_counter()
    T_st, L_st = st(A)
    X_st = T_st @ L_st @ L_st.T
    t_st = time.perf_counter() - t0

    t0 = time.perf_counter()
    T_ts, L_ts = ts(A)
    X_ts = np.linalg.inv(T_ts) @ L_ts @ L_ts.T
    t_ts = time.perf_counter() - t0

    t0 = time.perf_counter()
    Q_cj, G_cj, P_cj = cj(A)
    X_cj = Q_cj @ G_cj @ np.linalg.inv(P_cj)
    t_cj = time.perf_counter() - t0

    erro_chol = np.linalg.norm(A - X_chol, 2) / np.linalg.norm(A, 2)
    erro_qr   = np.linalg.norm(A - X_qr,   2) / np.linalg.norm(A, 2)
    erro_svd  = np.linalg.norm(A - X_svd,  2) / np.linalg.norm(A, 2)
    erro_st   = np.linalg.norm(A - X_st,   2) / np.linalg.norm(A, 2)
    erro_ts   = np.linalg.norm(A - X_ts,   2) / np.linalg.norm(A, 2)
    erro_cj   = np.linalg.norm(A - X_cj,   2) / np.linalg.norm(A, 2)

    table.append([
        n,
        fmt(condA),
        fmt(erro_chol), fmt(t_chol),
        fmt(erro_qr),   fmt(t_qr),
        fmt(erro_svd),  fmt(t_svd),
        fmt(erro_st),   fmt(t_st),
        fmt(erro_ts),   fmt(t_ts),
        fmt(erro_cj),   fmt(t_cj)
    ])

headers = [
    "n", "cond(A)",
    "erro_chol", "t_chol",
    "erro_qr", "t_qr",
    "erro_svd", "t_svd",
    "erro_st", "t_st",
    "erro_ts", "t_ts",
    "erro_cj", "t_cj"
]

print(tabulate(table, headers=headers, tablefmt="pretty", colalign=("right",)*len(headers)))

+-----+----------+-----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+
|   n |  cond(A) | erro_chol |   t_chol |  erro_qr |     t_qr | erro_svd |    t_svd |  erro_st |     t_st |  erro_ts |     t_ts |  erro_cj |     t_cj |
+-----+----------+-----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+
|   2 | 2.03e+00 |  9.10e-01 | 1.02e-04 | 0.00e+00 | 1.28e-04 | 6.65e-17 | 8.00e-05 | 1.41e-01 | 1.61e-04 | 1.33e-16 | 1.32e-04 | 2.04e-16 | 3.18e-03 |
|   3 | 1.78e+00 |  9.54e-01 | 9.03e-05 | 3.37e-17 | 1.56e-04 | 5.82e-16 | 8.02e-05 | 3.05e-01 | 2.21e-04 | 1.07e-02 | 2.06e-04 | 3.30e-16 | 2.45e-04 |
|   4 | 1.48e+00 |  9.68e-01 | 7.23e-05 | 2.32e-16 | 9.61e-05 | 5.33e-16 | 7.16e-05 | 2.43e-01 | 2.94e-04 | 1.05e-02 | 2.61e-04 | 2.54e-16 | 2.33e-04 |
|   6 | 1.79e+00 |  9.87e-01 | 7.89e-05 | 1.21e-16 | 1.00e-04 | 8.73e-16 | 8.55e-05 | 2.

A tabela resume o desempenho de diferentes métodos de decomposição para matrizes quadradas bem condicionadas. O método de **Cholesky**, na forma aplicada, apresenta erros elevados e não é adequado para reconstrução da matriz. **QR** e **SVD** mantêm erros próximos da precisão de máquina para todos os tamanhos, sendo o **QR** mais eficiente em tempo. Os métodos **ST** e **TS** têm erros intermediários e custo computacional crescente. O método **Conjugado** destaca-se por combinar alta precisão e bom desempenho, sendo a melhor alternativa fora dos métodos clássicos.

In [27]:
n_runs = [2, 3, 4, 6, 8, 10, 50, 100, 200, 500]
eps = 1e-6 # intensidade do ruído
np.random.seed(42)

def fmt(x):
    return f"{x:.2e}"

def chol_mp(A):
    R = np.linalg.cholesky(A.T @ A)
    return np.linalg.solve(R.T, np.linalg.solve(R, A.T)).T

def qr_mp(A):
    Q, R = np.linalg.qr(A, mode="reduced")
    return Q @ R

def svd_mp(A):
    U, S, Vt = np.linalg.svd(A, full_matrices=False)
    return U @ np.diag(S) @ Vt

table = []

for n in n_runs:

    A0 = np.random.rand(n, n)
    A0 = A0 + n * np.eye(n)

    # Ruído
    E = eps * np.random.randn(n, n)

    # Matriz ruidosa
    A = A0 + E

    condA = np.linalg.cond(A)

    t0 = time.perf_counter()
    X_chol = chol_mp(A)
    t_chol = time.perf_counter() - t0
    erro_chol = np.linalg.norm(A - X_chol, 2) / np.linalg.norm(A, 2)

    t0 = time.perf_counter()
    X_qr = qr_mp(A)
    t_qr = time.perf_counter() - t0
    erro_qr = np.linalg.norm(A - X_qr, 2) / np.linalg.norm(A, 2)

    t0 = time.perf_counter()
    X_svd = svd_mp(A)
    t_svd = time.perf_counter() - t0
    erro_svd = np.linalg.norm(A - X_svd, 2) / np.linalg.norm(A, 2)

    table.append([
        n,
        fmt(condA),
        fmt(erro_chol), fmt(t_chol),
        fmt(erro_qr),   fmt(t_qr),
        fmt(erro_svd),  fmt(t_svd),
    ])

headers = [
    "n", "cond(A)",
    "erro_chol", "t_chol",
    "erro_qr",   "t_qr",
    "erro_svd",  "t_svd",
]

print(tabulate(
    table,
    headers=headers,
    tablefmt="pretty",
    colalign=("right",) * len(headers)
))

+-----+----------+-----------+----------+----------+----------+----------+----------+
|   n |  cond(A) | erro_chol |   t_chol |  erro_qr |     t_qr | erro_svd |    t_svd |
+-----+----------+-----------+----------+----------+----------+----------+----------+
|   2 | 2.03e+00 |  9.10e-01 | 1.16e-04 | 1.90e-16 | 1.70e-04 | 3.59e-16 | 7.22e-05 |
|   3 | 1.72e+00 |  9.45e-01 | 5.84e-05 | 2.71e-16 | 9.67e-05 | 3.31e-16 | 5.08e-05 |
|   4 | 1.75e+00 |  9.72e-01 | 4.96e-05 | 2.53e-16 | 8.98e-05 | 8.14e-16 | 1.19e-04 |
|   6 | 1.58e+00 |  9.87e-01 | 5.29e-05 | 3.06e-16 | 1.19e-04 | 3.00e-15 | 1.69e-04 |
|   8 | 1.64e+00 |  9.93e-01 | 8.82e-05 | 2.67e-16 | 1.37e-04 | 1.14e-15 | 3.79e-04 |
|  10 | 1.65e+00 |  9.96e-01 | 1.18e-04 | 1.73e-16 | 1.00e-04 | 1.53e-15 | 1.37e-04 |
|  50 | 1.59e+00 |  1.00e+00 | 3.92e-04 | 4.80e-16 | 2.41e-04 | 7.42e-15 | 1.01e-03 |
| 100 | 1.56e+00 |  1.00e+00 | 1.16e-03 | 7.61e-16 | 1.15e-03 | 5.67e-15 | 4.12e-03 |
| 200 | 1.54e+00 |  1.00e+00 | 6.08e-03 | 6.64e-16 | 4

A inclusão de ruído aditivo não altera o condicionamento da matriz, mas afeta fortemente o desempenho dos métodos.

O método de **Cholesky** apresenta erros relativos próximos de 1 para todos os tamanhos, evidenciando forte amplificação do ruído devido ao uso das equações normais, tornando-o inadequado para reconstrução.

Os métodos **QR** e **SVD** mantêm erros da ordem da precisão de máquina, demonstrando alta robustez ao ruído. O **QR** é consistentemente mais rápido, enquanto o **SVD** é o mais custoso computacionalmente, especialmente para matrizes maiores.

Em síntese, com ruído presente, QR é a melhor escolha em termos de eficiência e estabilidade, e **SVD** é preferível apenas quando se deseja máxima robustez espectral.
